In [ ]:
 sos run /home/rl3328/xqtl-protocol/code/mnm_analysis/auto_finemapping_post_processing.ipynb  gwas_results_export      \
    --region_file /mnt/vast/hpc/homes/rf2872/codes/xqtl-analysis/resource/hg38_1362_blocks.bed     \
    --file_path  /home/rl3328/image_QTL/output/univariate_rss      \
    --name AI_IMAGE   \
    --suffix  univariate.rds       \
    --prefix  Automated_QC   \
    --min_corr 0.8    \
    --geno_ref /mnt/vast/hpc/csg/rf2872/data/Fungen_xQTL/geno_align/Fungen_xQTL.ROSMAP_NIA_WGS.ROSMAP_NIA_WGS.MSBB_WGS_ADSP_hg38.MiGA.MAP_Brain-xQTL_Gwas_geno_0.STARNET.aligned.bim.gz    \
    --context-meta /home/rl3328/image_GWAS/context_meta.tsv    \
    --cwd /home/rl3328/image_QTL/auto_finemapping_export \
    --gwas \
    --region-name chr3_162827771_166218666  \#optional 
    --step1_only #optional 

In [ ]:
[global]
import glob
import pandas as pd
# A region list file documenting the chr_pos_ref_alt of a susie_object
parameter: cwd = path("output")
parameter: name = "demo"

## Path to work directory where output locates
## Containers that contains the necessary packages
parameter: container = ""
import re
parameter: entrypoint= ""
# For cluster jobs, number commands to run per job
parameter: job_size = 50
# Wall clock time expected
parameter: walltime = "96h"
# Memory expected
parameter: mem = "6G"
# Number of threads
parameter: numThreads = 2
parameter: windows = 1000000
# use this function to edit memory string for PLINK input
from sos.utils import expand_size

In [ ]:
# Exporting cis susie_twas results
[cis_results_export_1, gwas_results_export_1]
# per chunk we process at most 200 datasets
parameter: per_chunk = 200
# Region list should have last column being region name. 
parameter:region_file=path()
# the path stored original output files
parameter:file_path=''
# assuming orinal files are named as prefix.id.suffix
# prefix of the original files (before id) to identify file.
parameter:prefix=[]
# suffix of the original files (after id) to identify file.
parameter:suffix=str
# if need to rename context, please load condition meta file
parameter: condition_meta = path()
# if the pip all variants in a cs < this threshold, then remove this cs
parameter: pip_thres = 0.05
# only keep top cs_size variants in one cs 
parameter: cs_size = 3
# provide exported meta to filter the exported genes 
parameter: exported_file = path()
# Optional: if a region list is provide the analysis will be focused on provided region. 
# The LAST column of this list will contain the ID of regions to focus on
# Otherwise, all regions with both genotype and phenotype files will be analyzed
parameter: region_list = path()
# Optional: if a region name is provided 
# the analysis would be focused on the union of provides region list and region names
parameter: region_name = []
# the (aligned) geno bim file to check allele flipping
parameter: geno_ref = path()
# context meta file to map renamed context backs
parameter: context_meta = path()
# default cretria in susie purity filtering, recommended to use 0.8 here 
parameter: min_corr = []
# set this parameter as `True` when exporting gwas data.
parameter: gwas = False
# set this parameter as `True` when exporting fsusie data.
parameter: fsusie = False
# set this parameter as `True` when exporting metaQTL data.
parameter: metaQTL = False
# set this parameter as `True` when exporting mnm data.
parameter: mnm = False
# flag this CS if cs_corr between this CS and top_cs < cs_corr_thres
parameter: cs_corr_thres = 0.5

import pandas as pd
import os

region = pd.read_csv(region_file, sep='\t', names=['chr', 'start', 'end', 'id'])

region_ids = []
# If region_list is provided, read the file and extract IDs
if not region_list.is_dir():
    if region_list.is_file():
        region_list_df = pd.read_csv(region_list, sep='\t', header=None, comment = "#")
        region_ids = region_list_df.iloc[:, -1].unique()  # Extracting the last column for IDs
    else:
        raise ValueError("The region_list path provided is not a file.")
        
if len(region_name) > 0:
    region_ids = list(set(region_ids).union(set(region_name)))
    
if len(region_ids) > 0:
    region = region[region['id'].isin(region_ids)]

# Function to create list of formatted strings for each row
def create_formatted_list(row):
    if len(prefix) > 0:
        formatted_list = [f"{file_path}/{p}.{row['id']}.{suffix}" for p in prefix]
    else:
        formatted_list = [f"{file_path}/{row['id']}.{suffix}"]  # GWAS data do not have prefix     
    return formatted_list

# Apply the function to each row
region['original_data'] = region.apply(create_formatted_list, axis=1)

def group_by_region(lst, partition):
    # from itertools import accumulate
    # partition = [len(x) for x in partition]
    # Compute the cumulative sums once
    # cumsum_vector = list(accumulate(partition))
    # Use slicing based on the cumulative sums
    # return [lst[(cumsum_vector[i-1] if i > 0 else 0):cumsum_vector[i]] for i in range(len(partition))]
    return partition

def filter_existing_paths(row):
    existing_paths = [path for path in row if os.path.exists(path)]
    return existing_paths

def is_file_exported(paths, results_set):
    for path in paths:
        basename = os.path.basename(path)
        if basename not in results_set:
            return False
    return True

region['original_data'] = region['original_data'].apply(filter_existing_paths)
region = region[region['original_data'].map(bool)]

# if provided exported meta, check if the original data are all exported already, isfo skip them 
if not exported_file.is_dir():
    if exported_file.is_file():
        results = pd.read_csv(exported_file, sep='\t')
        results_set = set(results['original_data'])
        results_set = {item.strip() for sub in results_set for item in sub.split(',')}
        mask = region['original_data'].apply(lambda paths: not is_file_exported(paths, results_set))
        region = region[mask]
    else:
        raise ValueError("The exported_file path provided is not a file.")

# added because there is frequently changed ID from upstream analysis!!!        
# If after filtering no region remains, update 'id' by concatenating 'chr' and original 'id' with '_'
if region.empty:
    print("No regions remain after filtering. Reattempting using concatenated id (chr_id).")
    # Update the 'id' column using the concatenation of the first column (chr) and the original 'id'
    region = pd.read_csv(region_file, sep='\t', names=['chr', 'start', 'end', 'id'])
    region['id'] = region['chr'] + "_" + region['id']
    
    # Reapply region_ids filter if provided
    if len(region_ids) > 0:
        region = region[region['id'].isin(region_ids)]
    
    # Recreate original_data with the updated id
    region['original_data'] = region.apply(create_formatted_list, axis=1)
    region['original_data'] = region['original_data'].apply(filter_existing_paths)
    region = region[region['original_data'].map(bool)]
    
    # Reapply exported_file filtering if applicable
    if not exported_file.is_dir():
        if exported_file.is_file():
            results = pd.read_csv(exported_file, sep='\t')
            results_set = set(results['original_data'])
            results_set = {item.strip() for sub in results_set for item in sub.split(',')}
            mask = region['original_data'].apply(lambda paths: not is_file_exported(paths, results_set))
            region = region[mask]
        else:
            raise ValueError("The exported_file path provided is not a file.")
regional_data = {
    'meta': [(row['chr'], row['start'], row['end'], row['id']) for _, row in region.iterrows()],
    'data': [(row['original_data']) for _, row in region.iterrows()]
}

meta_info = regional_data['meta']
stop_if(len(regional_data['data']) == 0, f' All files have been exported already')

input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta_info"
output: f"{cwd}/{name}_cache/{name}_{_meta_info[3]}.tsv"
task: trunk_workers = job_size, walltime = walltime, trunk_size = job_size, mem = mem, cores = numThreads, tags = f'{_output:bn}'
R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container = container, entrypoint=entrypoint
    library(tidyverse)
    library(data.table)
    library(susieR)
    library(pecotmr)
    # check top_loci existing or not
    has_rows <- function(df) {
      !is.null(df) && nrow(df) > 0
    }

    # function to allele flipping checking by mapping them to aligned genotype bim file
    align_to_genoref <- function(var_list, geno_ref, region ){
        geno_ref <- pecotmr:::tabix_region(file= geno_ref,
                    region = region)
        colnames(geno_ref) <- c('chr', 'pos', 'alt', 'ref')
        geno_ref <- geno_ref %>% mutate(chr = gsub('chr','',chr))
        var_list_df <- data.frame(chr = str_split(var_list,":|_",simplify = T)[,1] %>% gsub('chr','',.),
            pos = str_split(var_list,":|_",simplify = T)[,2],
            ref = str_split(var_list,":|_",simplify = T)[,3],
            alt = str_split(var_list,":|_",simplify = T)[,4])
        # merge_genotype_data from below cell
        aligned_var_df <- merge_genotype_data(geno_ref, var_list_df, all=FALSE)
        aligned_var <- aligned_var_df %>%
          mutate(id = {
            if (grepl(":", var_list[1])) {
              if (grepl("_", var_list[1])) {
                paste(chr, paste(pos, ref, alt, sep = "_"),sep = ':')
              } else {
                paste(chr, pos, ref, alt, sep = ":")
              }
            } else {
              paste(chr, pos, ref, alt, sep = "_")
            }
          }) %>%
          pull(id)
        if (grepl("chr", var_list[1]))  aligned_var <- paste0("chr",aligned_var)
        return(aligned_var)
    }
    
    # function to map variant list to geno ref
    merge_genotype_data <- function(df1, df2, all = TRUE) {
      setDT(df1)
      setDT(df2)
      df1[, key := paste(chr, pos, pmin(alt, ref), pmax(alt, ref))]
      df2[, key := paste(chr, pos, pmin(alt, ref), pmax(alt, ref))]
      df2[df1, on = "key", flip := i.alt == ref & i.ref == alt, by = .EACHI]
      df2[flip == TRUE, c("alt", "ref") := .(ref, alt)]
      if (all) {
        df_combined <- unique(rbindlist(list(df1[, .(chr, pos, alt, ref)], df2[, .(chr, pos, alt, ref)])), by = c("chr", "pos", "alt", "ref"))
      } else {
        df_combined <- df2[, .(chr, pos, alt, ref)]
      }
      return(df_combined)
    }

    # Function to filter credible sets with all variants PIP < 0.05 and update rows based on _min_corr suffix condition
    update_and_filter_cs_ids <- function(dat_con, dat_susie, df, purity_thresholds) {
      # Identify cs_coverage columns
      cs_columns <- grep("^cs_coverage", names(df), value = TRUE)
    
      # Flag rows with any cs_coverage > 0
      df$cs_all_non_zero_orig <- rowSums(df[cs_columns] == 0) != length(cs_columns)
    
      # Iterate over cs_coverage columns and their unique CS IDs
      for (cs_column in cs_columns) {
        unique_cs_ids <- unique(df[[cs_column]])
        # Update top loci based on minimum correlation for each coverage column
          for(purity_threshold in purity_thresholds){
              df <- update_top_loci_cs_annotation(dat_con, dat_susie, top_loci_df = df, coverage_value = cs_column, threshold = purity_threshold)
          }
    
        for (cs_id in unique_cs_ids[unique_cs_ids > 0]) {
          # Flag CS IDs where all PIPs < pip_thres (default 0.05)
          pip_check <- df[df[[cs_column]] == cs_id, "pip"] < 0.05
          # label the whole cluster if all pip < pip_thres (default 0.05)
          if (all(pip_check, na.rm = TRUE)) {
            df[[cs_column]] <- ifelse(df[[cs_column]] == cs_id, 0, df[[cs_column]])
          }
        }
      }
      # Identify min corr cs_coverage columns
      mincor_columns <- grep("_min_corr", names(df), value = TRUE) # Identify _mincor columns
    
      # Filter out rows where all cs_coverage columns are 0, all _mincor columns are 0,
      # and cs_all_non_zero_orig is TRUE
      df <- df %>%
        filter(!(cs_all_non_zero_orig &
                   rowSums(df[cs_columns] == 0) == length(cs_columns) &
                   rowSums(df[mincor_columns] == 0) == length(mincor_columns))) %>%
        select(-cs_all_non_zero_orig) # Remove the temporary flag column
      df <- df %>%
        select(-starts_with("cs_coverage"), all_of(cs_columns), all_of(mincor_columns))
      return(df)
     }
    
    
     # update top loci table
     update_top_loci_cs_annotation <- function(dat_con, dat_susie, top_loci_df, coverage_value, threshold ) {
       # update susie obj based CS
       if(coverage_value == 'cs_coverage_0.95') dat_susie_tmp <- dat_susie
        else dat_susie_tmp <- dat_susie$sets_secondary[[gsub('cs_','',coverage_value)]]
       #get purity res  
       purity_res <- dat_susie_tmp$sets$purity 
    
       not_pass_min_cs <- rownames(purity_res)[purity_res$min.abs.corr < threshold] %>%
         gsub('L', '', .)
       top_loci_df[[paste0( coverage_value, "_min_corr_", threshold)]] <- top_loci_df[[coverage_value]]
       if (length(not_pass_min_cs) > 0) {
         top_loci_df[[paste0( coverage_value, "_min_corr_", threshold)]][top_loci_df[[coverage_value]] %in% not_pass_min_cs] <- 0
       }
       return(top_loci_df)
     }
    
    # function to decide run update_and_filter_cs_ids or not
    process_top_loci <- function(dat_con, dat_susie,purity_thresholds) {
      data_frame <- dat_con$top_loci
      if (has_rows(data_frame)) {
        return(update_and_filter_cs_ids(dat_con, dat_susie, data_frame, purity_thresholds))
      } else {
        return(data_frame)
      }
    }

    # function to get pip with their variant name
    get_pip_withname <- function(susie_obj){
      pip = susie_obj$susie_result_trimmed$pip
      if(!(is.null(pip)))  names(pip) = susie_obj$variant_names
      return(pip)
    }
  
    # add column in top loci to indicate if the variant is identified in susie_on_top_pc top loci table or not
    annotate_susie <- function(obj, top_loci){
        tryCatch({
            df <- data.frame()
    
            results <- lapply(obj[['susie_on_top_pc']], function(x) {
              x[['top_loci']]
            })
            results <- results[!sapply(results, is.null)]
                              
            df <- bind_rows(results)
    
            top_loci_others <-  df %>%
              filter(rowSums(select(., starts_with('cs_coverage_'))) == 0)
            top_loci_cs <- df %>%
              filter(rowSums(select(., starts_with('cs_coverage_'))) > 0)
    
            susie_vars <- rbind(top_loci_others %>% filter(pip >= 0.05), top_loci_cs) %>% pull(variant_id)


            if(length(susie_vars) > 0) susie_vars <- align_to_genoref(var_list = susie_vars, geno_ref = geno_ref, region = paste0(gsub("chr","","${_meta_info[0]}"), ":", "${_meta_info[1]}", "-", "${_meta_info[2]}"))
    
            susie_vars_cs <- top_loci_cs %>% pull(variant_id)
            if(length(susie_vars_cs) > 0)  susie_vars_cs <- align_to_genoref(var_list = susie_vars_cs, geno_ref = geno_ref, region = paste0(gsub("chr","","${_meta_info[0]}"), ":", "${_meta_info[1]}", "-", "${_meta_info[2]}"))
    
            top_loci <- top_loci %>% mutate(annotated_susie = ifelse(variant_id %in% susie_vars, 1, 0),
                               annotated_susie_cs = ifelse(variant_id %in% susie_vars_cs, 1, 0))
    
            return(top_loci)
        }, error = function(e) {
            warning("Error in annotate_susie: ", e$message)
        })
    }
    
    # function to match context values and add analysis_name
    match_contexts <- function(df, context_meta) {
      # Split context_meta's context column into separate rows, one for each comma-separated value
      context_meta_expanded <- context_meta %>% 
        separate_rows(context, sep = ",") %>%
        mutate(context = trimws(context)) # Remove potential leading and trailing whitespace

      # Loop through each context in df to find the most precise match in context_meta_expanded
      df$super_context <- sapply(df$context, function(df_context) {
        # Find all potential matches
        potential_matches <- context_meta_expanded %>%
          filter(str_detect(df_context, context)) %>%
          arrange(desc(nchar(context))) # Sort potential matches by descending length of context

        # Select the longest match as the most precise one
        if(nrow(potential_matches) > 0) {
          return(potential_matches$context[1])
        } else {
          return(NA) # Return NA if no match is found
        }
      })

      return(df)
    }
    # Define a function to process context data and match contexts
    process_and_match_contexts <- function(context_meta_path, cons_top_loci, res_minp) {
      # Read context metadata
      context_meta <- fread(context_meta_path)

      # Extract super contexts
      super_contexts <- cons_top_loci[[1]] %>% unlist %>% names %>% as.character
      # Create a data frame of context and minimum p-values
      context_df <- data.frame(context = super_contexts,
                               min_p = res_minp[[1]][super_contexts] %>% unlist %>% as.numeric)
      # Match contexts using the previously defined match_contexts function
      context_df <- match_contexts(context_df, context_meta)
      # Filter context_df by super_context
      context_df_no_order <- context_df %>% filter(context == super_context)
      context_df_with_order <- context_df %>% filter(context != super_context) 

      return(list(context_df_no_order= context_df_no_order,context_df_with_order = context_df_with_order ))
    }

    # Function to filter introns based on leafcutter2 cluster
    process_lf2_cluster <- function(df){
        df <- df%>%
        mutate(cluster_id = str_extract(context, "clu_\\d+"), 
               category = str_extract(context, "([^:]+)(?=:EN)")) %>%
        group_by(cluster_id) %>%
        # filter the intron clusters with NE and IN events
        mutate(cluster_status = ifelse(any(category %in% c("NE", "IN")), "no", "yes")) %>%
        filter(cluster_status == "yes") %>%
        ungroup %>%
        filter(!str_detect(context, ":PR:")) %>%  #FIXME: only keep unproductive events in leafcutter2 results. 
        select(-cluster_status, -cluster_id, -category)
        return(df)
    }

    # Function to combine the contexted with and without order 
    combine_order <- function(context_df_no_order, context_df_with_order){
     context_df_with_order <- context_df_with_order %>%
        group_by(super_context) %>%
        summarise(min_p = min(min_p), .groups = 'keep') %>%
        left_join(context_df_with_order, by = c("super_context", "min_p" = "min_p"))

      # Combine context_df_no_order and context_df_with_order contexts
      cons_top_loci_minp <- c(context_df_no_order$context, context_df_with_order$context)
        return(cons_top_loci_minp)
    }
    
    # Revised summarise_lfsr_by_marker function with distinct error cases
    summarise_lfsr_by_marker <- function(multicontext_res,
                                           pos = NULL,
                                           markers = NULL,
                                           conditions = NULL,
                                           poslim = NULL,
                                           lfsr_cutoff = 0.01,
                                           sentinel_only = FALSE,
                                           cs_plot = NULL,
                                           conditional_effect = TRUE) {
      tryCatch({
        # Extract fitted object from multicontext_res
        fit <- multicontext_res[[1]]$mvsusie_fitted
    
        # Set default parameters if not provided
        if (is.null(markers)) markers <- fit$variable_names
        if (is.null(conditions)) conditions <- fit$condition_names
        if (is.null(pos)) pos <- seq(1, length(markers))
        if (is.null(poslim)) poslim <- range(pos)
        if (is.null(cs_plot)) cs_plot <- names(fit$sets$cs)
    
        # Create initial data frame for plotting
        pdat <- data.frame(pip = fit$pip, pos = pos, cs = as.character(NA),
                           stringsAsFactors = FALSE)
        css <- names(fit$sets$cs)
        for (i in css) {
          j <- fit$sets$cs[[i]]
          pdat[j, "cs"] <- i
        }
        rows <- which(!is.na(pdat$cs))
        pdat_cs <- pdat[rows, ]
    
        # Filter by position limits
        rows1 <- which(pdat$pos >= poslim[1] & pdat$pos <= poslim[2])
        rows2 <- which(pdat_cs$pos >= poslim[1] & pdat_cs$pos <= poslim[2])
        pdat <- pdat[rows1, ]
        pdat_cs <- pdat_cs[rows2, ]
    
        # Order and label credible sets
        pdat_cs$cs <- factor(pdat_cs$cs)
        css <- levels(pdat_cs$cs)
        L <- length(css)
        cs_pos <- sapply(fit$sets$cs[css], function(x) median(pos[x]))
        css <- css[order(cs_pos)]
        pdat_cs$cs <- factor(pdat_cs$cs, levels = css)
        cs_size <- sapply(fit$sets$cs[css], length)
        for (i in 1:L) {
          j <- css[i]
          if (cs_size[i] == 1) {
            levels(pdat_cs$cs)[i] <- sprintf("%s (1 SNP)", j)
          } else {
            levels(pdat_cs$cs)[i] <- sprintf("%s (%d SNPs, %0.3f purity)",
                                             j, cs_size[j], fit$sets$purity[j, "min.abs.corr"])
          }
        }
    
        # Set up traits and effects matrix
        traits <- conditions
        r <- length(traits)
        lmax <- nrow(fit$alpha)
        fit$b1_rescaled <- fit$b1_rescaled[, -1, ]
        rownames(fit$b1_rescaled) <- paste0("L", 1:lmax)
        rownames(fit$single_effect_lfsr) <- paste0("L", 1:lmax)
        colnames(fit$single_effect_lfsr) <- traits
        rownames(fit$alpha) <- paste0("L", 1:lmax)
        effects <- matrix(0, r, L)
        rownames(effects) <- traits
        colnames(effects) <- css
    
        # Initialize effect data frame
        effect_dat <- data.frame(matrix(as.numeric(NA),
                                          prod(length(conditions) * length(markers)), 8))
        names(effect_dat) <- c("trait", "marker", "pos", "effect", "z", "lfsr", "cs", "sentinel")
        effect_dat$trait <- rep(conditions, length(markers))
        effect_dat$marker <- rep(markers, each = length(conditions))
        effect_dat$pos <- rep(pos, each = length(conditions))
        effect_dat$sentinel <- 0
    
        # Process each credible set
        for (i in 1:L) {
          l <- css[i]
          j <- fit$sets$cs[[l]]
          b <- fit$b1_rescaled[l, j, ]
          if (conditional_effect) {
            b <- b / fit$alpha[l, j]
          }
          marker_names <- markers[j]
          marker_idx <- which(effect_dat$marker %in% marker_names)
          effect_dat[marker_idx, "cs"] <- l
          effect_dat[marker_idx, "lfsr"] <- rep(fit$single_effect_lfsr[l, ], length(marker_names))
          effect_dat[marker_idx, "effect"] <- as.vector(t(b))
          if (!is.null(fit$z)) {
            effect_dat[marker_idx, "z"] <- as.vector(t(fit$z[j, ]))
          }
          max_idx <- which.max(fit$alpha[l, j])
          effect_dat[which(effect_dat$marker == marker_names[max_idx]), "sentinel"] <- 1
          effects[, i] <- ifelse(is.null(nrow(b)), b, b[max_idx, ])
        }
    
        # Filter and process effect data
        effect_dat <- effect_dat[which(!is.na(effect_dat$cs)), ]
        rows1 <- which(effect_dat$pos >= poslim[1] & effect_dat$pos <= poslim[2])
        effect_dat <- effect_dat[rows1, ]
        effect_dat$marker_cs <- paste0(effect_dat$marker, "(", effect_dat$cs, ")")
        pdat_sentinel <- effect_dat[which(effect_dat$sentinel == 1), ]
        pdat_sentinel <- unique(pdat_sentinel[, c("marker", "marker_cs", "pos")])
        pdat_sentinel$pip <- fit$pip[match(pdat_sentinel$marker, fit$variable_names)]
    
        # Optionally keep only sentinel variants
        if (sentinel_only) {
          effect_dat <- effect_dat[which(effect_dat$sentinel == 1), ]
        }
        # Optionally filter by specified credible sets to plot
        if (!missing(cs_plot)) {
          effect_dat <- effect_dat[which(effect_dat$cs %in% cs_plot), ]
        }
        effect_dat$cs <- factor(effect_dat$cs, levels = css)
        effect_dat$trait <- factor(effect_dat$trait, traits)
        rows <- which(effect_dat$lfsr < lfsr_cutoff)
        effect_dat <- effect_dat[rows, ]
    
        # Error Case 1: No variants left after LFSR cutoff filtering
        if (nrow(effect_dat) == 0) {
          message("Warning Case 1: No variants passed the LFSR cutoff threshold; returning NA values.")
          top_loci <- multicontext_res[[1]][['top_loci']]
          top_loci$trait <- NA
          top_loci$lfsr <- NA
          return(top_loci)
        }
    
        effect_dat <- effect_dat %>% 
          mutate(pip = fit$pip[match(marker, names(fit$pip))],
                 variant_id = marker)
    
        # Summarise results and merge with top loci
        result <- effect_dat %>%
          group_by(variant_id, cs) %>%
          arrange(trait) %>%
          summarise(trait = paste(trait, collapse = ";"), 
                    lfsr = paste(lfsr, collapse = ";"),
                    .groups = "drop") %>% 
          mutate(cs_coverage_0.95 = gsub('L', '', cs)) %>%
          select(-cs)
    
        top_loci <- multicontext_res[[1]][['top_loci']]
        merge(top_loci, result, by = c('variant_id', 'cs_coverage_0.95'), all.x = TRUE)
      }, error = function(e) {
        # Error Case 2: Any unexpected error in processing
        message("Warning Case 2: An unexpected error occurred: ", e$message)
        top_loci <- tryCatch(multicontext_res[[1]][['top_loci']],
                             error = function(x) data.frame(variant_id = NA, cs_coverage_0.95 = NA))
        top_loci$trait <- NA
        top_loci$lfsr <- NA
        return(top_loci)
      })
    }
    
    # function to add effect size
    add_coef_column <- function(dat_study) {
      # Ensure the required components exist
      # Check if dat_study contains both required components
    if (!("susie_result_trimmed" %in% names(dat_study)) || !("top_loci" %in% names(dat_study))) {
      # Return a null list instead of stopping
      return(data.frame())
    }
          
      # Extract the coefficient matrix and ensure top_loci is a data.frame
      coef_matrix <- dat_study[["susie_result_trimmed"]][["coef"]][-1,] / dat_study[["susie_result_trimmed"]][['pip']]
      dt <- dat_study[["top_loci"]]
      if (!is.data.frame(dt)) {
        dt <- as.data.frame(dt)
      }
      
      # Check that dt has the required columns
      if (!all(c("variant_id", "trait") %in% names(dt))) {
        stop("The 'top_loci' data frame must contain 'variant_id' and 'trait' columns.")
      }
      
      # Add the 'coef' column by matching each variant with its corresponding traits
      dt[["conditional_effect"]] <- mapply(function(variant, trait_str) {
        # Return NA if trait_str is missing or variant not found in coef_matrix
        if (is.na(trait_str) || !(variant %in% rownames(coef_matrix))) {
          return(NA_character_)
        }
        # Split the trait string by ';' and trim extra whitespace
        traits <- trimws(unlist(strsplit(trait_str, ";")))
        # Subset coefficients for the variant and specified traits
        coefs <- coef_matrix[variant, traits, drop = TRUE]
        paste(coefs, collapse = ";")
      }, dt[["variant_id"]], dt[["trait"]], USE.NAMES = FALSE)
      
      dat_study[["top_loci"]] <- dt
      return(dt)
    }

    ###### MAIN ######
    # Process each path and collect results
    orig_files = c(${",".join(['"%s"' % x.absolute() for x in _input])})
     # Extract info from each RDS file
    results <- list()
    gene = "${_meta_info[3]}"
    geno_ref <- "${geno_ref}"

    # Post Processing: Extracting info
    # use for loop instead of apply to save memory
    res <- res_sum <- res_minp <- cons_top_loci <- list()
    pip_sum <- data.frame()
    for(i in seq_along(orig_files)) {
      rds_path <- orig_files[i]
       dat <- tryCatch({
        readRDS(rds_path)
      }, error = function(e) {
        writeLines(rds_path %>% basename, gsub(".tsv","_error","${_output}"))
        return(NULL) # 
      })

      if(is.null(dat)) next
  
      #extract qtl type from susie rds file name, if we have set decent condtiton name, this could be removed 
      temp_list <- list() # Temporary list to store inner results
      genes <- names(dat)
        for(id in genes){
          dat_study <- dat[[id]]
          temp_list <- list()
          if(${"TRUE" if mnm else "FALSE"}) {
              if (length(dat[[id]]) <= 1) {
                message("No multi-contexts results due to no twas results; skipping multi-context export.")
                next
              }
              conditions <- paste(orig_files[i] %>% basename %>% str_split(., '[.]', simplify = T) %>% .[,1],dat_study[['context_names']] %>% paste(  ., collapse = ';'), sep = ':') 
              dat_study[['top_loci']] <- summarise_lfsr_by_marker(dat)
              dat_study[['top_loci']] <- add_coef_column(dat_study)
            } else conditions <- names(dat_study)[sapply(dat_study, function(x) length(x) > 0)]
  
          if(length(conditions) > 0) {
            for(condition in conditions) {
                if (${"FALSE" if mnm else "TRUE"}) dat_con <- dat_study[[condition]]${[['fsusie_summary']] if fsusie else ''} else dat_con <- dat_study${[['fsusie_summary']] if fsusie else ''}
                if (${"TRUE" if gwas else "FALSE"}){
                  method <- names(dat_study[[condition]])[!names(dat_study[[condition]]) %in% c('rss_data_analyzed', 'diagnostics')] ## FIXME: this method is not showing in meta (not need if only one method). or we can use `context@method` in meta
                  # Make an automated decision on which method to use
                  if (!is.null(dat_con$diagnostics) && nrow(dat_con$diagnostics) > 0) {  # Check if there's diagnostic table for this condition
                    # Identify high correlation columns for top_cs
                    top_cs_index <- which.max(abs(dat_con$diagnostics$top_z))
                    # Get the column names that match the pattern "cs_corr_" followed by numbers
                    cs_corr_cols <- grep("^cs_corr_\\d+$", names(dat_con$diagnostics), value = TRUE)
                    # Initialize an empty list to store high correlation columns
                    high_corr_cols <- list()
                    # Iterate through the correlation columns and check for correlations > cs_corr_thres
                    for (col in cs_corr_cols) {
                      if (!is.na(dat_con$diagnostics[top_cs_index, ..col]) && abs(dat_con$diagnostics[top_cs_index, ..col]) > ${cs_corr_thres}) {
                        high_corr_cols <- c(high_corr_cols, col)
                      }
                    }
                    # Process the data
                    diagnostic_decision <- auto_decision(dat_con$diagnostics, high_corr_cols)
                    method <- if (unique(diagnostic_decision$method) == "BCR") {
                      message(paste0("BCR for ", condition, " study and ", gene, " block"))
                      "bayesian_conditional_regression_RSS_QC_RAISS_imputed"
                    } else if (unique(diagnostic_decision$method) == "SER") {
                      message(paste0("SER for ", condition, " study and ", gene, " block"))
                      "single_effect_NO_QC"
                    } else if (unique(diagnostic_decision$method) == "BVSR") {
                      message(paste0("BVSR for ", condition, " study and ", gene, " block"))
                      "susie_rss_RSS_QC_RAISS_imputed"
                    } else {
                      stop("Invalid method selected.")
                    }
                  }
                  if (length(method) != 1) {
                    stop("Multiple methods after automated QC.Pls check!")
                  }
                  dat_con <- dat_con[[method]]
                }
                dat_susie <- dat_con$susie_result_trimmed
                # calculate the sum of pip for the vairnats with pip > 0
                pip_sum = rbind(pip_sum, 
                                data.frame(pip_sum = dat_susie[['pip']] [dat_susie[['pip']] > 0] %>% sum, 
                                           condition = condition)
                               )
                                
                # rename context names if needed
                if(${"TRUE" if condition_meta.is_file() else "FALSE"}){
                      meta <- suppressMessages(read_delim("${condition_meta}", col_names = F))
                      context <- meta %>% filter(X1 == condition) %>% pull(X2)
                      if (length(context) == 0) {
                        context <- condition
                        message("No matching entries found. context has been set to the condition value.")
                      }
                    } else {context <- condition}
                  
                # align variants to aligned geno
                if(has_rows(dat_con$top_loci) || has_rows(dat_con$preset_top_loci)) cons_top_loci[[id]][[context]] <- context else  cons_top_loci[[id]][[context]] <- NULL    
                variant_ids <- c(dat_con$top_loci$variant_id, dat_con$variant_names, dat_con$preset_variants_result$top_loci$variant_id, dat_con$preset_variants_result$variant_names)
                unique_variant_ids <- unique(variant_ids)
                aligned_variant_ids <- align_to_genoref(unique_variant_ids, geno_ref, paste0(gsub("chr","","${_meta_info[0]}"), ":", "${_meta_info[1]}", "-", "${_meta_info[2]}"))
                names(aligned_variant_ids) <- unique_variant_ids

                # change beta or z in top loci and sumstats
                top_loci_changed_indexes <- which(dat_con$top_loci$variant_id !=  aligned_variant_ids[dat_con$top_loci$variant_id] %>% as.character )
                if(has_rows(dat_con$top_loci) & length(top_loci_changed_indexes) > 0) {
                      dat_con$top_loci$conditional_effect[top_loci_changed_indexes] <- (-1)* dat_con$top_loci$conditional_effect[top_loci_changed_indexes] # FIXME if coef does not need to check flip
                      if (${"FALSE" if gwas else "TRUE"}) {
                          dat_con$top_loci$betahat[top_loci_changed_indexes] <- (-1)* dat_con$top_loci$betahat[top_loci_changed_indexes]
                      } else {
                          dat_con$top_loci$z[top_loci_changed_indexes] <- (-1)* dat_con$top_loci$z[top_loci_changed_indexes]
                      }
                }
                all_changed_indexes <- which(dat_con$variant_names !=  aligned_variant_ids[dat_con$variant_names] %>% as.character )
                if(length(all_changed_indexes) > 0) {
                      if (${"FALSE" if gwas else "TRUE"}) {
                          dat_con$sumstats$betahat[all_changed_indexes] <- (-1)* dat_con$sumstats$betahat[all_changed_indexes]
                      } else {
                           dat_con$sumstats$z[all_changed_indexes] <- (-1)* dat_con$sumstats$z[all_changed_indexes]
                      }
                }

                # change variant names 
                if(has_rows(dat_con$top_loci)) dat_con$top_loci$variant_id <- aligned_variant_ids[dat_con$top_loci$variant_id] %>% as.character ###
                dat_con$variant_names <- aligned_variant_ids[dat_con$variant_names] %>% as.character ###

                
                if (${"FALSE" if gwas else "TRUE"}) { 
                    res[[id]][[context]] <- list(
                      top_loci = process_top_loci(dat_con, dat_susie, purity_thresholds = c(${",".join(['"%s"' % x for x in min_corr])})),
                      pip = get_pip_withname(dat_con)
                    )
                    # add conditional effect size
                    if(${"FALSE" if mnm else "TRUE"} & has_rows(dat_con[['top_loci']])) dat_con[['top_loci']] <- dat_con[['top_loci']] %>% mutate(conditional_effect = (coef(dat_susie)/ (dat_susie[['pip']]))[variant_id])
                    # the preset data in fsusie is actually from first PC and analyzed by susie, we'd like to remove them to avoid misleading
                    # although no preset res in mnm, we can put it here with the same layer structure. It would not report preset results
                    if (${"FALSE" if fsusie else "TRUE"}) {
                          if(has_rows(dat_con$preset_variants_result$top_loci)) {
                              dat_con$preset_variants_result$top_loci$variant_id <- aligned_variant_ids[dat_con$preset_variants_result$top_loci$variant_id] %>% as.character
                              # Calculate and add the conditional effect in one step
                              if(has_rows(dat_con$preset_variants_result[['top_loci']])){
                                  dat_con$preset_variants_result[['top_loci']] <- dat_con$preset_variants_result[['top_loci']] %>%
                                      mutate(conditional_effect = (coef(dat_con$preset_variants_result$susie_result_trimmed) /
                                                                     dat_con$preset_variants_result$susie_result_trimmed[['pip']])[variant_id])
                                  dat_con$top_loci$conditional_effect[top_loci_changed_indexes] <- (-1)* dat_con$top_loci$conditional_effect[top_loci_changed_indexes] # FIXME if coef does not need to check flip
                                }
                              }
                          dat_con$preset_variants_result$variant_names <- aligned_variant_ids[dat_con$preset_variants_result$variant_names] %>% as.character
                          # change beta in preset top loci
                          preset_top_loci_changed_indexes <- which(dat_con$preset_variants_result$top_loci$variant_id !=  aligned_variant_ids[dat_con$preset_variants_result$top_loci$variant_id] %>% as.character )
                          if(has_rows(dat_con$top_loci) & length(preset_top_loci_changed_indexes) > 0) dat_con$preset_variants_result$top_loci$betahat[preset_top_loci_changed_indexes] <- (-1)* dat_con$preset_variants_result$top_loci$betahat[preset_top_loci_changed_indexes]   

                          res[[id]][[context]][['region_info']] = dat_con$region_info
                          res[[id]][[context]][['CV_table']] = dat_con$twas_cv_result$performance
                          res[[id]][[context]][['preset_top_loci']] = process_top_loci(dat_con$preset_variants_result, dat_con$preset_variants_result$susie_result_trimmed, purity_thresholds = c(${",".join(['"%s"' % x for x in min_corr])}))
                          res[[id]][[context]][['preset_pip']] = get_pip_withname(dat_con$preset_variants_result)
                    } else {
                          res[[id]][[context]][['region_info']] = dat_study[[condition]]$region_info
                          res[[id]][[context]][['CV_table']] = dat_study[[condition]]$twas_cv_result$performance
                          res[[id]][[context]][['top_loci']] = annotate_susie(dat_study[[condition]], res[[id]][[context]][['top_loci']])
                          
                    } 
                # fsusie do not have sumstats or p value
                if (${"FALSE" if fsusie or mnm else "TRUE"}) {
                    res_sum[[id]][[context]] <- list(
                      variant_names = dat_con$variant_names,
                      sumstats = dat_con$sumstats
                    )

                    if (${"FALSE" if fsusie or mnm else "TRUE"}) res_minp[[id]][[context]] <- min(pecotmr:::wald_test_pval(dat_con$sumstats$betahat, dat_con$sumstats$sebetahat,  n = 1000)) ##assuming sample size is 1000
                }
  
                } else {
                    res[[id]][[context]][[method]] <- list(
                      top_loci = process_top_loci(dat_con, dat_susie, purity_thresholds = c(${",".join(['"%s"' % x for x in min_corr])})),
                      pip = get_pip_withname(dat_con)
                    )
                    res_sum[[id]][[context]][[method]] <- list(
                      variant_names = dat_con$variant_names,
                      sumstats = dat_con$sumstats
                    )

                }
                

                if(has_rows(dat_con$top_loci) || has_rows(dat_con$preset_top_loci)) cons_top_loci[[id]][[context]] <- context else  cons_top_loci[[id]][[context]] <- NULL
              }
            }
        }
    }

    cons_top_loci <- cons_top_loci %>% compact()  # Use 'compact' to remove NULLs
    cons_top_loci <- if(length(cons_top_loci) > 0) cons_top_loci else NA

    combine_data = combine_data_sumstats = cons_top_loci_minp = ''
    combine_data = paste0("${_output:add}","/","${name}", ".", ${'"epigenomics_"' if fsusie else '""'}, ${ '"metabolomics_"' if metaQTL else '""'}, gene, ".cis_results_db.export.rds")
    if (${"FALSE" if fsusie else "TRUE"}) combine_data_sumstats = gsub("export.rds$", "export_sumstats.rds", combine_data)
  
    if (${"TRUE" if exported_file.is_file() else "FALSE"}){
        if (file.exists(combine_data)) {
          res_exp <- readRDS(combine_data)
          res[[gene]] <- c(res[[gene]], res_exp[[gene]]) # this may need to change if tehre are multiple genes in one rds file. 
        }
        if (file.exists(combine_data_sumstats)) {
          res_sum_exp <- readRDS(combine_data_sumstats)
          res_sum[[gene]] <- c(res_sum[[gene]], res_sum_exp[[gene]])
        }
    }
    saveRDS(res, combine_data)
    # only save sumstats results when NOT fsusie or multi gene mvsusie
    if (${"FALSE" if fsusie or mnm else "TRUE"}) saveRDS(res_sum, combine_data_sumstats)
  
    # generate md5 for data transferring
    system(paste("md5sum ",combine_data, " > ", paste0(combine_data, ".md5")))
    if (${"FALSE" if fsusie or mnm else "TRUE"}) system(paste("md5sum ",combine_data_sumstats, " > ", paste0(combine_data_sumstats, ".md5")))
  
      
    TSS <- tryCatch({dat_con$region_info$region_coord$start},  error = function(e) {return(NA)})
    if (length(res) > 0) conditions = paste(names(res[[1]]), collapse = ",") else conditions = ''
    
    # fsusie does not have sumstats or pvalue, do not need to run this

    # if (${"FALSE" if fsusie or gwas or mnm else "TRUE"})  {
    #     context_map <- process_and_match_contexts('${context_meta}', cons_top_loci, res_minp)
    #     context_map$context_df_with_order <- process_lf2_cluster(context_map$context_df_with_order)
    #     cons_top_loci_minp <- combine_order(context_map$context_df_no_order, context_map$context_df_with_order)
    # }

    # save pip sum file 
     write_delim(pip_sum, 
                 paste0("${_output:add}","/","${name}", ".", ${'"epigenomics_"' if fsusie else '""'}, ${ '"metabolomics_"' if metaQTL else '""'}, gene, ".pip_sum"), 
                 delim = '\t')

    meta = data.frame(chr="${_meta_info[0]}", start="${_meta_info[1]}", end="${_meta_info[2]}", region_id="${_meta_info[3]}", TSS =  if(is.null(TSS)) NA else TSS, 
                      original_data = paste(basename(orig_files), collapse = ","), combined_data = basename(combine_data), combined_data_sumstats = basename(combine_data_sumstats), 
                      conditions = conditions, 
                      conditions_top_loci = if(length(cons_top_loci) > 0) cons_top_loci[[1]] %>% unlist %>% names %>% as.character %>% paste(., collapse = ',') else ''
                      # , conditions_top_loci_minp = if(length(cons_top_loci_minp) > 0) cons_top_loci_minp %>% paste(., collapse = ',') else ''
                      )

    write_delim(meta, "${_output}", delim = '\t')
  

In [ ]:
[cis_results_export_2, gwas_results_export_2]
# set it as True if you only want run first step (running parallel) and combine them manually after all finished 
parameter: step1_only = False
skip_if(step1_only)
# provide exported meta to filter the exported genes 
parameter: exported_file = path()
# optional: qtl or gwas, there is slightly different in qtl and gwas rds file
parameter: gwas = False
input: group_by = 'all'
output: f"{cwd}/{name}.{'block_results_db' if gwas else 'cis_results_db'}.tsv" 
# stop_if(_input[0] not in locals().keys(), 'All files have been exported already') #FIXME should we remove to a separate file. sothat we can stop globally as above
task: trunk_workers = 1, walltime = '1h', trunk_size = 1, mem = '16G', cores = 1, tags = f'{_output:bn}'
bash: expand = "${ }", container = container, stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', entrypoint=entrypoint
    if [ -e "${_output:ad}/${name}_cache/" ]; then
        sed 's/^chr/#chr/' `ls ${_output:ad}/${name}_cache/*tsv |head -n1` | head -n 1 > ${_output:an}.temp
        tail -n +2 -q ${_output:ad}/${name}_cache/*.tsv >> ${_output:an}.temp
        error_files=$(find "${_output:ad}/${name}_cache/" -type f -name "*_error")

        if [[ -n $error_files ]]; then
            cat $error_files >> ${_output:an}.error_genes
        else
            echo "No truncated files detected"
        fi
    else
        echo "All files have been exported already"
    fi
    
R: expand = "${ }", container = container, stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', entrypoint=entrypoint
    if (file.exists(paste0(${_output:anr},".temp"))) {
        library(tidyverse)
        meta <- read_delim(paste0(${_output:anr},".temp"), delim = '\t')

        if (${"TRUE" if exported_file.is_file() else "FALSE"}){
          exp_meta <- read_delim(${exported_file:r}, delim = '\t')
          meta <- bind_rows(meta, exp_meta) %>%
              group_by(`#chr`, start, end, region_id, TSS) %>%
              summarise(across(c(original_data, combined_data, combined_data_sumstats, conditions, conditions_top_loci), 
                               ~paste(unique(.), collapse = ",")),
                        .groups = 'drop')
              }

        write_delim(meta, ${_output:r}, delim = '\t')
    }

In [ ]:
[cis_results_export_3]
# set it as True if you only want run first step (running parallel) and combine them manually after all finished 
parameter: step1_only = False

skip_if(step1_only)
bash: expand = "${ }", container = container, stderr = f'{_input:n}.stderr', stdout = f'{_input:n}.stdout', entrypoint=entrypoint   
    rm -rf "${_input:ad}/${name}_cache/"
    rm -rf ${_input:an}.temp

In [ ]:
[export_top_loci]
parameter: export_path=path
parameter: region = str
parameter: prefix = str
parameter: suffix = str
parameter: fsusie_prefix = ''
parameter: preset_top_loci = False
parameter: lfsr_thres = 0.01
input: f"{export_path}/{prefix}.{fsusie_prefix}{region}.{suffix}" 
output: f"{cwd:a}/{prefix}.{region}.toploci.bed.gz" 
task: trunk_workers = 1, walltime = '1h', trunk_size = job_size, mem = '16G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }", container = container, stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', entrypoint=entrypoint
    library(tidyverse)
    library(data.table)
    lfsr_thres <- as.numeric(${lfsr_thres})
    #function to filter lfsr
    filter_lfsr <- function(df, lfsr_thres){
        df_na <- df %>% filter(is.na(lfsr)|lfsr == '') %>% mutate(event_ID = NA)
        df %>%
            mutate(
              lfsr_list = str_split(lfsr, ";") %>% map(as.numeric),
              effect_list = str_split(conditional_effect, ";"),
              event_list = str_split(event_ID, ";"),
              valid_indices = map(lfsr_list, ~ which(.x < lfsr_thres))
            ) %>%
            rowwise() %>%
            mutate(
              event_ID = paste(event_list[valid_indices], collapse = ";"),
              conditional_effect = paste(effect_list[valid_indices], collapse = ";"),
              lfsr = paste(lfsr_list[valid_indices], collapse = ";")
            ) %>%
            filter(event_ID != "") %>%
            select(any_of(colnames(file_top_loci_rename))) %>%
            ungroup() %>% rbind(df, df_na)
        }
    
    ## main
    res <- readRDS("${_input}")
    file_top_loci <- lapply(names(res), function(gene) {
        lapply(names(res[[gene]]), function(context) {
          lapply(c("top_loci"${',"preset_top_loci"' if preset_top_loci else ''}), function(method) {
            finemap_method <- names(res[[gene]][[context]])
            if (!(method %in% finemap_method)) {# for rss_susie
              if (length(finemap_method) == 1) { # that is for one model 
                temp_df <- res[[gene]][[context]][[finemap_method]][[method]]
              } else {
                stop("Not the only finemap model, models are ", finemap_method)
              }
              if (!is.null(temp_df)) {
                mutate(temp_df, study = context, method = method, finemap_method = finemap_method, region = gene)
              } else {
                NULL  
              }
            } else {
              temp_df <- res[[gene]][[context]][[method]]
              if (!is.null(temp_df)) {
                mutate(temp_df, study = context, method = method, region = gene)
              } else {
                NULL  
              }
            }
          })
        }) %>% bind_rows()  
    }) %>% bind_rows() 
    if (nrow(file_top_loci) == 0 || is.null(file_top_loci)) {
      warning("No top loci found in the finemapping rds.")
    } else {
      file_top_loci_rename <- file_top_loci %>% 
        # Select necessary columns from the original table
        select(any_of(c(
          "variant_id", "pip", 
          "cs_coverage_0.95_min_corr","cs_coverage_0.7_min_corr", "cs_coverage_0.5_min_corr",
          "cs_coverage_0.95_min_corr_0.8","cs_coverage_0.7_min_corr_0.8", "cs_coverage_0.5_min_corr_0.8",
          "cs_coverage_0.95_min_corr_0.5","cs_coverage_0.7_min_corr_0.5", "cs_coverage_0.5_min_corr_0.5",
          "study", "method", "region", "finemap_method", "conditional_effect", "lfsr", "trait"
        ))) %>% 
        mutate(
          # Split variant_names into separate components
          chr = str_split(variant_id, ":", simplify = TRUE)[, 1],  # Chromosome
          pos = str_split(variant_id, ":", simplify = TRUE)[, 2],  # Position
          a1  = str_split(variant_id, ":", simplify = TRUE)[, 4],  # Allele 1
          a2  = str_split(variant_id, ":", simplify = TRUE)[, 3],  # Allele 2
          # Rename and duplicate columns as required
          variant_ID = variant_id,
          gene_ID    = region,
          event_ID   = study,
          PIP = pip,
          finemap_model = finemap_method,
        )  %>%
        {
          if (any(c("cs_coverage_0.95_min_corr_0.8", "cs_coverage_0.7_min_corr_0.8", "cs_coverage_0.5_min_corr_0.8", 
                    "cs_coverage_0.95_min_corr_0.5", "cs_coverage_0.7_min_corr_0.5", "cs_coverage_0.5_min_corr_0.5", 
                    "cs_coverage_0.95_min_corr", "cs_coverage_0.7_min_corr", "cs_coverage_0.5_min_corr") %in% colnames(.))) {
            mutate(.,
              cs_coverage_0.95 = if ("cs_coverage_0.95_min_corr_0.8" %in% colnames(.)) get("cs_coverage_0.95_min_corr_0.8") else 
                                if ("cs_coverage_0.95_min_corr" %in% colnames(.)) get("cs_coverage_0.95_min_corr") else NA,
              
              cs_coverage_0.7 = if ("cs_coverage_0.7_min_corr_0.8" %in% colnames(.)) get("cs_coverage_0.7_min_corr_0.8") else 
                                if ("cs_coverage_0.7_min_corr" %in% colnames(.)) get("cs_coverage_0.7_min_corr") else NA,
              
              cs_coverage_0.5 = if ("cs_coverage_0.5_min_corr_0.8" %in% colnames(.)) get("cs_coverage_0.5_min_corr_0.8") else 
                                if ("cs_coverage_0.5_min_corr" %in% colnames(.)) get("cs_coverage_0.5_min_corr") else NA,
              
              cs_coverage_0.95_purity0.5 = if ("cs_coverage_0.95_min_corr_0.5" %in% colnames(.)) get("cs_coverage_0.95_min_corr_0.5") else NA,
              cs_coverage_0.7_purity0.5 = if ("cs_coverage_0.7_min_corr_0.5" %in% colnames(.)) get("cs_coverage_0.7_min_corr_0.5") else NA,
              cs_coverage_0.5_purity0.5 = if ("cs_coverage_0.5_min_corr_0.5" %in% colnames(.)) get("cs_coverage_0.5_min_corr_0.5") else NA
            )
          } else .
        } %>%
          # Conditionally add lfsr if the column exists
        mutate(region_id = basename("${_input}") %>% str_split(., '[.]', simplify = T) %>% .[,2])  %>% 
        # Conditionally add lfsr if the column exists
        { if ("lfsr" %in% colnames(.)) mutate(., lfsr = lfsr, context = str_split(event_ID, ':', simplify = T) %>% .[,1]) else . } %>% 
        { if ("lfsr" %in% colnames(.) & lfsr_thres != 0) mutate(., event_ID = trait) else . } %>% 
        # Select and order the final set of columns
        select(any_of(c("chr", "pos", "a1", "a2", "variant_ID", "gene_ID", "event_ID", "cs_coverage_0.95", "cs_coverage_0.7", "cs_coverage_0.5",
                        "cs_coverage_0.95_purity0.5", "cs_coverage_0.7_purity0.5", "cs_coverage_0.5_purity0.5", "PIP", "conditional_effect", 
                        "lfsr", "region_id", "context"${'",method"' if preset_top_loci else ''},"finemap_model"))) %>% 
        # Sort by chromosome and position (convert pos to numeric if necessary)
        arrange(chr, as.numeric(pos))
      # Check if "lfsr" is in column names
      if ("lfsr" %in% colnames(file_top_loci_rename) & lfsr_thres != 0) file_top_loci_rename <- filter_lfsr(file_top_loci_rename, lfsr_thres = lfsr_thres)
      fwrite(file_top_loci_rename,"${_output}")
    }
    